# Standard Quantum Tomography
Quara supports several estimators (linear, least squares, maximum-likelihood) for standard quantum tomography of state, POVM, and gate. Here we briefly explain how to use them with examples on 1-qubit system. Quara supports much larger systems. However, performances (computation speed and constraint feasibility) of numerical optimization solver implemented are quite low, and we do not recommend to perform tomographic data-processing for systems larger than 1-qubit. Improvements on the performances are necessary tasks at the development of Quara in the near future.

In [1]:
import numpy as np

# quara
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.tester_typical import (
    generate_tester_states,
    generate_tester_povms,
)
from quara.objects.qoperation_typical import generate_qoperation
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt

from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.loss_function.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.minimization_algorithm.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)

## Quantum State Tomography (1-qubit)
First, we consider quantum state tomography on 1-qubit system. We prepare a system.

In [2]:
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
print(c_sys)

elemental_systems:
[0] 0 (system_id=140214303147968)

dim: 2
basis:
(array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.j        ]]), array([[ 0.70710678+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.70710678+0.j]]))


In order to perform data-pocessing of standard quantum tomography, we need some pre-knowledge information on the tomographic experiment. Suppose that we performed projective measurements along with x, y, and z axes. We call them a tester. 

In [3]:
# Testers
names = ["x", "y", "z"]
testers = generate_tester_povms(c_sys=c_sys, names = names)
for i, tester in enumerate(testers):
    print("Tester ", i, ":\n", tester)

Tester  0 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]]
Tester  1 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.         -0.70710678  0.        ]]
Tester  2 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


We generate a StandardQst object. We set seed for generating pseudo-data. The seed is unnecessary if you do not generate pseudo-data.

In [4]:
seed = 7896
qst = StandardQst(testers, on_para_eq_constraint=True, schedules="all", seed_data=seed)

Suppose that the state of the system to be estimated is $|A\rangle$. We call it the true state.

In [5]:
mode = "state"
name = "a"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
State

Dim:
2

Vec:
[0.70710678 0.5        0.5        0.        ]


The true probability distribution is given as follows.

In [6]:
prob_dists = qst.calc_prob_dists(true)
print(prob_dists)

[[0.85355339 0.14644661]
 [0.85355339 0.14644661]
 [0.5        0.5       ]]


Suppose that we performed tomographic experiment and repeated each sub-experiment with tester element 1000 times.

In [7]:
#qst.reset_seed()
num_data = 1000
empi_dists = qst.generate_empi_dists(state=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.866, 0.134]))
(1000, array([0.849, 0.151]))
(1000, array([0.503, 0.497]))


Here empi_dists is a set of empirical distributions, which is a pair of the repetition number and relative frequencies.

When we choose a linear estimator, the estimate is calculated as follows.

In [8]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("is estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[0.70710678 0.51760216 0.49356053 0.00424264]
is estimate physical? :  False

Eigenvalues are:  [1.005733131206568, -0.005733131206568459]


An eigenvalue of the estimated density matrix is negative, which violates the requirement of positive-semidefiniteness on density matrix. This kind of violation can occur when we choose a linear estimator. In order to avoid the problem, we need to perform a constraint optimization at the data-processing.

When we choose a constraint least squares estimator, the estimate is calculated as folllows. Here we choose a projected gradient descent back-tracking as the numerical optimization algorithm.

In [9]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError()
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()
algo_option = ProjectedGradientDescentBacktrackingOption(mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable", num_history_stopping_criterion_gradient_descent=1)

result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[0.70710678 0.51173451 0.48796542 0.00419455]

is estimate physical? :  False

Eigenvalues are:  [1.0000000252534356, -2.5253435731453777e-08]


An eigenvalue of the estimated density matrix is negative as same as observed at the case of linear estimator, although the strength of the violation is reduced from O(10^{-4}) to O(10^{-8}). This is due to the low constraint feasibility of the numerical solver implemented at the current version of Quara, which must be improved in the near future.

## POVM tomography (1-qubit)
Next, we consider POVM tomography on 1-qubit system.

In [10]:
# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names = ["x0", "y0", "z0", "z1"]
testers = generate_tester_states(c_sys=c_sys, names = names)
povmt = StandardPovmt(testers, num_outcomes=2, on_para_eq_constraint=True, schedules="all")

In [11]:
mode = "povm"
name = "z"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


In [12]:
prob_dists = povmt.calc_prob_dists(true)
print(prob_dists)

[[0.5 0.5]
 [0.5 0.5]
 [1.  0. ]
 [0.  1. ]]


In [13]:
num_data = 1000
empi_dists = povmt.generate_empi_dists(povm=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.481, 0.519]))
(1000, array([0.472, 0.528]))
(1000, array([1., 0.]))
(1000, array([0., 1.]))


In [14]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678 -0.02687006 -0.03959798  0.70710678]
 [ 0.70710678  0.02687006  0.03959798 -0.70710678]]

is estimate physical? :  False

Eigenvalues are: [[1.0011436919686807, -0.0011436919686808854], [1.0011436919686814, -0.001143691968680663]]


In [15]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(4)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.7071068  -0.02674827 -0.03941851  0.70550034]
 [ 0.70710677  0.02674827  0.03941851 -0.70550034]]

is estimate physical? :  False

Eigenvalues are: [[1.000000020995048, 2.2410805849970572e-13], [0.9999999999997758, -2.0995047856709262e-08]]


## Quantum Process Tomography (1-qubit)
Finally, we consider quantum process tomography on 1-qubit system. 

In [16]:
# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names_states = ["x0", "y0", "z0", "z1"]
testers_states = generate_tester_states(c_sys=c_sys, names = names_states)
names_povms = ["x", "y", "z"]
testers_povms = generate_tester_povms(c_sys=c_sys, names=names_povms)
qpt = StandardQpt(states=testers_states, povms=testers_povms, on_para_eq_constraint=True, schedules="all")

In [17]:
mode = "gate"
name = "hadamard"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Gate

Dim:
2

HS:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]]


In [18]:
prob_dists = qpt.calc_prob_dists(true)
print(prob_dists)

[[0.5 0.5]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [0.5 0.5]]


In [19]:
num_data = 1000
empi_dists = qpt.generate_empi_dists(gate=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.478, 0.522]))
(1000, array([0.533, 0.467]))
(1000, array([1., 0.]))
(1000, array([0.52, 0.48]))
(1000, array([0., 1.]))
(1000, array([0.511, 0.489]))
(1000, array([1., 0.]))
(1000, array([0.493, 0.507]))
(1000, array([0.487, 0.513]))
(1000, array([0., 1.]))
(1000, array([0.507, 0.493]))
(1000, array([0.514, 0.486]))


In [20]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

Type:
Gate

Dim:
2

HS:
[[ 1.     0.     0.     0.   ]
 [ 0.    -0.044  0.04   1.   ]
 [ 0.     0.066 -1.    -0.014]
 [ 0.001  0.999  0.021 -0.027]]

is estimate physical? :  False

Eigenvalues are: [-0.03627627 -0.02101806  0.05672322  2.00057111]


In [21]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError()
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

Type:
Gate

Dim:
2

HS:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.02656507e-03 -1.09288050e-02  1.77136570e-02  9.84091419e-01]
 [-2.51796302e-03  5.18102148e-02 -9.76171254e-01 -1.85715203e-03]
 [ 9.98824726e-04  9.78244503e-01  3.60986493e-02 -1.15642106e-02]]

is estimate physical? :  False

Eigenvalues are: [-1.49033317e-08 -1.28148015e-08  2.97230730e-02  1.97027695e+00]
